In [10]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"..","..","Algorithm encapsulation"))
from SimAnalysis import SimAnalysis
from LMband import LMband
from Multiband import Multiband
from MXband import MXband
from MPbandTool import MPbandTool
from MPband import MPband


In [11]:
def get_rf(d,p):
    tmp=[]
    for i,a in enumerate(p):
        idx=np.where(a!=0)[0]
        a=a&0
        if len(idx)>0:
            a[:idx[0]]=1
        tmp.append((d[i]*a).sum())
    return tmp

In [12]:
phase = np.loadtxt("../data/phase")
cir=phase[0].sum()
phase=phase/cir
vol = np.loadtxt("../data/volume")
qv = np.loadtxt("../data/path_vol")
phase_get = np.load("../data/phase_get.npy")
d = np.loadtxt("../data/distance")
straight_get = np.load("../data/straight_get.npy")
ison = np.array([0, 0, 0, 1, 1, 1])
px = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
pxb = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
qb=[120,120]
lower, upper = 8, 14
linspace = np.arange(lower, upper, 1)
lin_num = len(linspace)

# 南北直行方向绿灯时间
sg=np.array([(straight_get[i]*phase).sum(axis=1) for i in range(len(straight_get))])
rho = vol[0] / vol[1]
num = len(vol[0])
numr = len(qv)
nump = len(phase_get)
M = 1e6
nx = 0.00000001
be = 8
speedScope = np.array([lower, upper])
speedVar = np.array([-2, 2])

g=np.array([(phase_get[i]*phase).sum(axis=1) for i in range(numr) ])
r=1-g
rf=np.array([get_rf(phase,phase_get[i]) for i in range(numr)])

ex=[1,0,1,0,1,0,1,1,1,1,1,0]
dwt=15
qb_x=np.ones_like(sg)*20
cap=np.ones_like(sg,dtype=int)*3

spd_on=pd.read_csv(os.path.join(os.getcwd(),"..","..","speed analysis","上行速度数据.csv"),index_col=0)
spd_in=pd.read_csv(os.path.join(os.getcwd(),"..","..","speed analysis","下行速度数据.csv"),index_col=0)
p=[0,0,0,0,1,0,0,0,1,0,1,0,0]

In [13]:
def sim_start(df,name,sta):
    sim=SimAnalysis(df,phase,ex,os.path.join(os.getcwd(),"..","..","vissim","aa","zhongsan.inp"),
                os.path.join(os.getcwd(),"..","..","vissim","aa","vissim.ini"),
                os.path.join(os.getcwd(),"..","..","vissim","aa","zhongsan - 副本.inp"))
    sim.set_saturation(sta)
    sim.start()
    delayx=sim.analysis_delay(os.path.join(os.getcwd(),"..","..","vissim","aa",'zhongsan - 副本.vlz'))
    tvltx=sim.analysis_traveltime(os.path.join(os.getcwd(),"..","..","vissim","aa",'zhongsan - 副本.rsz'))
    delayx.to_csv("./staturation/delay_{}.csv".format(name),encoding="utf-8-sig")
    tvltx.to_csv("./staturation/traveltime_{}.csv".format(name),encoding="utf-8-sig")

In [14]:
def LMband_sensitivity(cycle,sta,dwt,tpy,idx):
    lmb=LMband(phase, [cycle[0],cycle[1]], vol*sta, qv*sta, phase_get, d, straight_get, ison, px, pxb, qb,qb_x,cap, lower, upper, 
           linspace, be, speedVar[0], speedVar[1],ex, dwt,lower,upper,speedVar[0], speedVar[1],spd_on,spd_in)
    lmb.set_M2_time_limit(60*15)
    lmb.solve()
    df=lmb.get_dataframe()
    sim_start(df,"{}_{}_{}".format("LMB",tpy,idx),sta)

In [15]:
def Multiband_sensitivity(cycle,sta,dwt,tpy,idx):
    mb=Multiband(phase,[cycle[0],cycle[1]],straight_get,p,d,lower,upper,speedVar[0], speedVar[1],vol,px)
    mb._solve()
    a=mb.get_dataframe()
    mpb=MPbandTool(phase, [cycle[0],cycle[1]], vol*sta, qv*sta, phase_get, d, straight_get, ison, px, pxb, qb, lower, upper, 
            be, speedVar[0], speedVar[1],ex, dwt,lower,upper,speedVar[0], speedVar[1],p,a.offset/a.z,
            1/a.z,np.array([a.t1/a.z,a.t2/a.z]))
    mpb._solve()
    df=mpb.get_dataframe()
    df["dw1"]=np.ones_like(df.z)*dwt
    df["dw2"]=np.ones_like(df.z)*dwt
    sim_start(df,"{}_{}_{}".format("MB",tpy,idx),sta)
    

In [16]:
def MXband_sensitivity(cycle,sta,dwt,tpy,idx):
    mb=MXband(phase,[cycle[0],cycle[1]],straight_get,d,lower,upper,speedVar[0], speedVar[1],vol,px)
    mb._solve()
    a=mb.get_dataframe()
    mpb=MPbandTool(phase, [80,180], vol*sta, qv*sta, phase_get, d, straight_get, ison, px, pxb, qb, lower, upper, 
                be, speedVar[0], speedVar[1],ex, dwt,lower,upper,speedVar[0], speedVar[1],a.p,a.offset/a.z,
                1/a.z,np.array([a.t1/a.z,a.t2/a.z]))
    mpb._solve()
    df=mpb.get_dataframe()
    df["dw1"]=np.ones_like(df.z)*dwt
    df["dw2"]=np.ones_like(df.z)*dwt
    sim_start(df,"{}_{}_{}".format("MXB",tpy,idx),sta)

In [17]:
def MPband_sensitivity(cycle,sta,dwt,tpy,idx):
    mpb=MPband(phase, [cycle[0],cycle[1]], vol*sta, qv*sta, phase_get, d, straight_get, ison, px, pxb, qb, lower, upper, 
            be, speedVar[0], speedVar[1],ex, dwt,lower,upper,speedVar[0], speedVar[1],p)
    mpb._solve()
    df=mpb.get_dataframe()
    sim_start(df,"{}_{}_{}".format("MPB",tpy,idx),sta)

In [18]:
def cycle_analysis():
    #周期灵敏度分析
    for i in [80, 100, 120, 140,160, 180]:
        LMband_sensitivity([i,i],1,15,"cycle",i)
        Multiband_sensitivity([i,i],1,15,"cycle",i)
        MXband_sensitivity([i,i],1,15,"cycle",i)
        MPband_sensitivity([i,i],1,15,"cycle",i)
cycle_analysis()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Tried aggregator 2 times.
MIP Presolve eliminated 467 rows and 117 columns.
MIP Presolve modified 949 coefficients.
Aggregator did 52 substitutions.
Reduced MIP has 817 rows, 436 columns, and 2734 nonzeros.
Reduced MIP has 68 binaries, 76 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (3.07 ticks)
Probing fixed 0 vars, tightened 51 bounds.
Probing time = 0.00 sec. (0.31 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 244 coefficients.
Reduced MIP has 817 rows, 436 columns, and 2734 nonzeros.
Reduced MIP has 68 binaries, 76 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.53 ticks)
Probing time = 0.00 sec. (0.29 ticks)
Clique table members: 29.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using 

In [19]:
def dwelltime_analysis():
    for i in [10, 12, 14, 16, 18, 20]:
        LMband_sensitivity([80,180],1,i,"dwt",i)
        Multiband_sensitivity([80,180],1,i,"dwt",i)
        MXband_sensitivity([80,180],1,i,"dwt",i)
        MPband_sensitivity([80,180],1,i,"dwt",i)
dwelltime_analysis()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Tried aggregator 2 times.
MIP Presolve eliminated 467 rows and 117 columns.
MIP Presolve modified 949 coefficients.
Aggregator did 52 substitutions.
Reduced MIP has 817 rows, 436 columns, and 2734 nonzeros.
Reduced MIP has 68 binaries, 76 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (3.07 ticks)
Probing fixed 0 vars, tightened 51 bounds.
Probing time = 0.00 sec. (0.31 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 244 coefficients.
Reduced MIP has 817 rows, 436 columns, and 2734 nonzeros.
Reduced MIP has 68 binaries, 76 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.53 ticks)
Probing time = 0.00 sec. (0.29 ticks)
Clique table members: 29.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using 

AttributeError: 'NoneType' object has no attribute 'solve_details'

In [ ]:
def staturation_analysis():
    for i in [0.4,0.6,0.8, 1, 1.2, 1.4,1.6]:
        LMband_sensitivity([80,180],i,15,"sta",i)
        Multiband_sensitivity([80,180],i,15,"sta",i)
        MXband_sensitivity([80,180],i,15,"sta",i)
        MPband_sensitivity([80,180],i,15,"sta",i)
staturation_analysis()